In [1]:
import pandas as pd
from pandas import Timestamp
from pandas.tseries.offsets import BDay
import numpy as np
from scipy.interpolate import interp1d
import os
import re
import seaborn as sns
import matplotlib.pyplot as plt
from pandas.tseries.offsets import BDay

# 1. Change directory to raw data folder

In [2]:
# Get the current working directory
current_directory = os.getcwd()

# Get the parent directory by going one level up
parent_directory = os.path.dirname(current_directory)

# Get the path of raw data folder
data_raw_folder = os.path.join(parent_directory, 'data/raw')

# Change the current working directory to raw data folder
os.chdir(data_raw_folder)

# 2. Import datasets

In [3]:
# New issues data
new_issues = pd.read_csv('new_issues.csv', index_col=0)
new_issues_characteristics =  pd.read_csv('new_issues_characteristics.csv', index_col=0)
new_issues_prices_refinitive_ask =  pd.read_csv('new_issues_prices_askyield_refinitive.csv', index_col=0)
swap_rates = pd.read_csv('swap_rates.csv', index_col=0)

In [4]:
# Comparable bonds data
chunk_dfs = []
for i in range(6):
    chunk_df = pd.read_csv(f'comparable_bonds_peers_duration_iboxx_{i+1}.csv', index_col=0)
    chunk_dfs.append(chunk_df)

comparable_bonds_iboxx = pd.concat(chunk_dfs, ignore_index=True)

In [5]:
# Index data
iboxx_indices = pd.read_csv('iboxx_indices.csv', sep=';')
move_index = pd.read_csv('move_index.csv')

# 3. Modify raw datasets and create subsets

## 3.1 New issues

In [6]:
# Create new issues dataframe with selected columns
new_issues_sliced = new_issues.loc[:,['securities_isin', 'ticker', 'dealDate',
                                      'issuerType', 'paymentType', 'countryName', 'COUNTRY_ISO_CODE', 
                                      'moodys', 'sp', 'fitch', 'expectedSize', 'actualSize', 'minimumDenomination', 
                                      'securityType', 'maturityTerm', 'coupon', 'seniority_name_1', 
                                      'seniority_name_1_adj', 'esgType', 'referenceType', 'ipt_reference',
                                      'ipt_price_down', 'ipt_price_up', 'guidance_reference', 'guidance_price_down',
                                      'guidance_price_up', 'launchValue', 'reofferValue', 'yieldToMaturity', 'duration',
                                     'Industry_Group', 'Industry_Sector']]

# Change time type
new_issues_sliced['dealDate'] = pd.to_datetime(new_issues_sliced['dealDate'])
new_issues_sliced['dealDate'] = new_issues_sliced['dealDate'].dt.floor('d')

## 3.2 Comparable bonds

In [7]:
# Change time type
comparable_bonds_iboxx['date'] = pd.to_datetime(comparable_bonds_iboxx['date'])
comparable_bonds_iboxx['date'] = comparable_bonds_iboxx['date'].dt.floor('d')

In [8]:
# Rename column
comparable_bonds_iboxx = comparable_bonds_iboxx.rename(columns={'new_issue_isin': 'isin', 'ticker': 'tickerCompBond'})

## 3.3 New issues characteristics; New issues prices

In [9]:
# Merge new issues with corresponding payment ranks
new_issues_sliced = new_issues_sliced.merge(new_issues_characteristics, left_on = 'securities_isin', right_on = 'security', how = 'inner')
new_issues_sliced = new_issues_sliced.drop(['security'], axis=1)

In [10]:
# Change time type in refinitive prices dataset
new_issues_prices_refinitive_ask['Date'] = pd.to_datetime(new_issues_prices_refinitive_ask['Date'])
new_issues_prices_refinitive_ask['Date'] = new_issues_prices_refinitive_ask['Date'].dt.date
new_issues_prices_refinitive_ask['Date'] = new_issues_prices_refinitive_ask['Date'].astype('datetime64')
new_issues_prices_refinitive_ask['Date'] = new_issues_prices_refinitive_ask['Date'].dt.floor('d')

In [11]:
# Filter out 'wrong' prices (EUR price instead of yield)
new_issues_prices_refinitive = new_issues_prices_refinitive_ask[(abs(new_issues_prices_refinitive_ask['Ask Yield']) < 10) | (new_issues_prices_refinitive_ask['Ask Yield'].isna())]

## 3.4 Iboxx indices

In [12]:
# Change time type in iboxx index dataset
iboxx_indices['Download_Date'] = pd.to_datetime(iboxx_indices['Download_Date'])

In [13]:
# Create issue dataframe with selected columns
iboxx_indices_sliced = iboxx_indices.loc[:,['Download_Date', 'Name', 'Annual_Yield_to_Maturity', 'Expected_Remaining_Life']]

In [14]:
# Keep only iBoxx € Corporates Senior and iBoxx € Financials Senior
iboxx_indices_sliced = iboxx_indices_sliced[iboxx_indices_sliced['Name'].isin(['iBoxx € Corporates Senior', 'iBoxx € Financials Senior'])]

## 3.5 Swap rates

In [15]:
# Extract the column names and their corresponding years
columns = swap_rates.columns
years = []

# Determine if the column represents months or years and extract the corresponding number
for col in columns:
    match = re.search(r'(\d+)([YM])=', col)
    if match:
        num = int(match.group(1))
        unit = match.group(2)
        if unit == 'Y':
            years.append(num)
        elif unit == 'M':
            years.append(num / 12)
            
# Create a dictionary to map the old column names to the new column names
new_columns = {col: yr for col, yr in zip(columns, years)}

# Rename the columns in the DataFrame
swap_rates.rename(columns=new_columns, inplace=True)

# Sort the columns in ascending order
swap_rates = swap_rates.reindex(sorted(swap_rates.columns), axis=1)

# Change time type in refinitive prices dataset
swap_rates = swap_rates.reset_index()
swap_rates['Date'] = pd.to_datetime(swap_rates['Date'])
swap_rates['Date'] = swap_rates['Date'].dt.date
swap_rates['Date'] = swap_rates['Date'].astype('datetime64')
swap_rates['Date'] = swap_rates['Date'].dt.floor('d')
swap_rates = swap_rates.set_index('Date')

In [16]:
# Extract the maturities from column names
maturities = [col for col in swap_rates.columns]

# Interpolate the swap rate curve for each timestamp
interpolated_swap_rates = {}
for timestamp, row in swap_rates.iterrows():
    swap_rate_interpolator = interp1d(maturities, row, kind='cubic')
    interpolated_swap_rates[timestamp] = swap_rate_interpolator

# 4. Analysis

In [17]:
# Set display options
pd.set_option('display.float_format', lambda x: '%.2f' % x)

# Extract year from dealDate
new_issues_sliced['year'] = pd.to_datetime(new_issues_sliced['dealDate']).dt.year

# Define the desired order of payment ranks
payment_rank_order = ['Secured', 'Sr Preferred', 'Sr Non Preferred', 'Sr Unsecured']

# Group by year and Industry_Sector, calculate average percentage and total number of bonds
grouped = new_issues_sliced.groupby(['year', 'Industry_Sector']).size().unstack()
distribution = grouped.div(grouped.sum(axis=1), axis=0) * 100
average_percentage = distribution.mean()
total_bonds = grouped.sum(axis=1)

# Get the top 5 sectors based on highest average percentage
top_sectors = average_percentage.nlargest(5).index.tolist()

# Group by year and PAYMENT_RANK, calculate percentage distribution
grouped_rank = new_issues_sliced.groupby(['year', 'PAYMENT_RANK']).size().unstack()
distribution_rank = grouped_rank.div(grouped_rank.sum(axis=1), axis=0) * 100
distribution_rank = distribution_rank[payment_rank_order].round(2).astype(str) + '%'

# Filter the distribution and averages dataframes based on the top sectors
filtered_distribution = distribution[top_sectors]
filtered_distribution = filtered_distribution.round(2).astype(str) + '%'

filtered_averages = new_issues_sliced.groupby('year').agg({'coupon': 'mean', 'maturityTerm': 'mean',
                                                            'AMT_ISSUED': 'mean', 'MIN_INCREMENT': 'mean',
                                                          'duration': 'mean'})

# Divide AMT_ISSUED column by 1 million
filtered_averages['AMT_ISSUED'] /= 1000000

# Divide MIN_INCREMENT column by 1000
filtered_averages['MIN_INCREMENT'] /= 1000

# Merge the filtered distribution, total bonds, and averages dataframes
table = pd.concat([filtered_distribution, distribution_rank, total_bonds, filtered_averages], axis=1)

# Transpose the table
table = table.transpose()

# Rename the columns for clarity
table.columns = table.columns.astype(str)

# Print the table
print(table)

year                      2017    2018    2019    2020    2021    2022    2023
Financial               55.74%  65.75%  64.52%  49.09%  61.14%  68.07%  67.31%
Utilities                 8.2%  10.27%   7.62%   12.2%  11.14%  10.26%   10.9%
Consumer, Non-cyclical   6.56%    8.9%   9.38%   12.8%   10.6%   6.29%   8.33%
Consumer, Cyclical       4.92%   7.53%   4.69%    6.4%   6.25%   5.83%   5.13%
Communications          11.48%   2.74%   7.33%   4.57%   2.72%    2.1%   1.92%
Secured                 32.79%   41.1%  28.74%   18.9%   23.1%  37.53%   39.1%
Sr Preferred              nan%   6.85%  10.26%   7.32%  11.14%  11.19%   8.97%
Sr Non Preferred         4.92%   6.85%  12.32%   8.54%   8.42%   6.06%   10.9%
Sr Unsecured             62.3%  45.21%  48.68%  65.24%  57.34%  45.22%  41.03%
0                        61.00  146.00  341.00  328.00  368.00  429.00  156.00
coupon                    1.04    1.04    0.66    0.88    0.40    2.27    3.72
maturityTerm              9.30    7.58    7.74    7.

## 3.6 Create dataframe of initial MS of new issues

In [18]:
# MS spread from IPREO data
new_issues_yield_ipreo = new_issues_sliced.loc[:, ['dealDate', 'securities_isin', 'ticker', 'maturityTerm', 'duration', 'PAYMENT_RANK', 'issuerType', 'reofferValue']]
new_issues_yield_ipreo = new_issues_yield_ipreo.rename(columns={'securities_isin': 'isin'})
new_issues_yield_ipreo = new_issues_yield_ipreo.drop_duplicates()

In [19]:
# Keep new issues with known initial MS
new_issues_initial_prices = new_issues_yield_ipreo.loc[~new_issues_yield_ipreo['reofferValue'].isna(), :]

In [20]:
new_issues_initial_prices = new_issues_initial_prices.reset_index(drop=True)

## 3.7 Find comparable bonds at the day of the issue

In [21]:
# Merge comparable bonds on 'ticker' and 'dealDate' columns
new_issues_with_comp = new_issues_initial_prices.merge(comparable_bonds_iboxx, left_on=['isin', 'dealDate'], right_on=['isin', 'date'], how='inner')
new_issues_with_comp = new_issues_with_comp.drop_duplicates()

In [22]:
# Keep comparable bonds with the same payment rank
new_issues_with_comp = new_issues_with_comp[new_issues_with_comp['PAYMENT_RANK']==new_issues_with_comp['Payment_Rank']]
new_issues_with_comp = new_issues_with_comp.reset_index(drop=True)

In [23]:
# new_issues_with_comp_filtered[['dealDate', 'ISIN']].drop_duplicates().to_csv('comparable_bonds_peers.csv')

In [24]:
# Calculate the maturity difference
new_issues_with_comp['duration_diff'] = new_issues_with_comp['duration'] - new_issues_with_comp['Duration']
new_issues_with_comp['maturity_diff'] = new_issues_with_comp['maturityTerm'] - new_issues_with_comp['Time_To_Maturity']
new_issues_with_comp['maturity_diff_abs'] = abs(new_issues_with_comp['maturityTerm'] - new_issues_with_comp['Time_To_Maturity'])

In [25]:
# Keep bonds with close maturities (difference < 1 year)
new_issues_with_comp_filtered = new_issues_with_comp.loc[new_issues_with_comp['maturity_diff_abs'] <= 0.5]
new_issues_with_comp_filtered = new_issues_with_comp_filtered.reset_index(drop=True)

In [26]:
# Convert 'dealDate' to datetime type
new_issues_with_comp_filtered['dealDate'] = pd.to_datetime(new_issues_with_comp_filtered['dealDate'])

# Extract year from 'dealDate'
new_issues_with_comp_filtered['year'] = new_issues_with_comp_filtered['dealDate'].dt.year

# Group by 'year', 'isin', and calculate count, average duration, and average maturity_diff_abs for each group
grouped_data_all = new_issues_with_comp_filtered.groupby(['year', 'isin']).agg({
    'ticker': 'count',  # Rename 'count' to 'ticker'
    'duration_diff': 'mean',
    'maturity_diff': 'mean'
}).reset_index()
grouped_data_all.rename(columns={'ticker': 'count'}, inplace=True)  # Rename the 'ticker' column back to 'count'

# Calculate the average count of matched bonds for each year
average_count_per_year = grouped_data_all.groupby('year')['count'].mean().reset_index()
average_count_per_year.rename(columns={'count': 'Average_Count'}, inplace=True)

# Create dummy columns to indicate if 'ticker' and 'tickerCompBond' are the same or not
new_issues_with_comp_filtered['same_ticker'] = new_issues_with_comp_filtered['ticker'] == new_issues_with_comp_filtered['tickerCompBond']

# Group by 'year', 'isin', and the dummy column, and calculate sum for each group
grouped_data = new_issues_with_comp_filtered.groupby(['year', 'isin', 'same_ticker']).size().reset_index(name='count')

# Calculate the total count of matched bonds for each year
total_count_per_year = grouped_data.groupby('year')['count'].sum().reset_index()

# Calculate the number of unique ISINs for each year
unique_isins_per_year = grouped_data.groupby('year')['isin'].nunique().reset_index()

# Calculate the average count of matched bonds per ISIN for each year with the same and different tickers
average_count_per_year_same = grouped_data[grouped_data['same_ticker'] == True].groupby('year')['count'].sum().reset_index()
average_count_per_year_same['count'] /= unique_isins_per_year['isin']
average_count_per_year_same.rename(columns={'count': 'Average_Same_Tickers'}, inplace=True)

average_count_per_year_not_same = grouped_data[grouped_data['same_ticker'] == False].groupby('year')['count'].sum().reset_index()
average_count_per_year_not_same['count'] /= unique_isins_per_year['isin']
average_count_per_year_not_same.rename(columns={'count': 'Average_Diff_Tickers'}, inplace=True)

# Calculate the average duration for each year, grouped by ISINs
average_duration_per_year = grouped_data_all.groupby('year')['duration_diff'].mean().reset_index()
average_duration_per_year.rename(columns={'duration_diff': 'Average_Duration'}, inplace=True)

# Calculate the average maturity_diff_abs for each year, grouped by ISINs
average_maturity_diff_abs_per_year = grouped_data_all.groupby('year')['maturity_diff'].mean().reset_index()
average_maturity_diff_abs_per_year.rename(columns={'maturity_diff': 'Average_Maturity_Diff_Abs'}, inplace=True)

# Create DataFrames with the average counts for each year
average_count_per_year_summary = average_count_per_year.set_index('year').transpose().reset_index(drop=True)
average_count_per_year_summary.columns.name = None  # Remove the index name

# Combine all DataFrames into a single DataFrame
result_df = pd.concat([
    average_count_per_year_summary,
    average_count_per_year_same.set_index('year').transpose(),
    average_count_per_year_not_same.set_index('year').transpose(),
    average_duration_per_year.set_index('year').transpose(),
    average_maturity_diff_abs_per_year.set_index('year').transpose()
], axis=0)

# Rename the index names for clarity
result_df.rename(index={0: 'Average'}, inplace=True)

# Display the final DataFrame with 5 rows and years as columns
print(result_df)

                           2017  2018  2019  2020  2021  2022  2023
Average                    5.39  7.33  8.27  6.27  7.02 13.48 16.38
Average_Same_Tickers       0.18  0.22  0.26  0.35  0.27  0.53  0.56
Average_Diff_Tickers       5.21  7.11  8.01  5.91  6.75 12.95 15.81
Average_Duration           0.27  0.08  0.21  0.18  0.21 -0.19 -0.35
Average_Maturity_Diff_Abs  0.07  0.04  0.08  0.05  0.05  0.02  0.03
